In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import cufflinks as cf
from scipy.integrate import odeint
from math import exp
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
#Import thermodynamic data
therm = pd.read_excel('properties database.xlsx',sheetname='Thermwatprops',index_col='T')
therm.head()

,P,ro,SV,Cp,sH
T,,,,,
4,0.9,1000.0,1.0,4.205,NaN
5,0.9,1000.0,1.0,4.202,0.075
10,1.2,999.8,1.0,4.192,0.150
15,1.7,999.2,1.0,4.18551),0.223
20,2.3,998.3,1.0,4.182,0.296


In [3]:
#Import steam thermo data
steam = pd.read_excel('properties database.xlsx',sheetname='Steamprops',index_col='T')
steam.head()

,P,SV,ro,HL,HV,HG,sHS
T,,,,,,,
3.8,0.8,160.00,0.00626,15.8,2493,2509,9.058
17.5,2.0,67.00,0.01490,73.5,2460,2534,8.725
32.9,5.0,28.20,0.03540,137.8,2424,2562,8.396
45.8,10.0,14.70,0.06820,191.8,2393,2585,8.151
60.1,20.0,7.65,0.13100,251.5,2358,2610,7.909


In [4]:
#IMport water thermo data 
wat = pd.read_excel('properties database.xlsx',sheetname='Fluidwatprops', index_col="T")
wat.head()

,P,dU,kU,alpha,HL,NPr
T,,,,,,
0.01,0.6,1.78,1.792,-0.070,0.0,13.67
5.00,0.9,1.52,NaN,0.016,21.0,NaN
10.00,1.2,1.31,1.304,0.088,41.9,9.47
15.00,1.7,1.14,NaN,0.151,62.9,NaN
20.00,2.3,1.00,1.004,0.207,83.8,7.01


In [5]:
#Import units data
units = pd.read_excel('properties database.xlsx',sheetname='units')
units

,Fluid props,Units,Steam Props,Units.1,Water Therm Props,Units.2
0,T,(oC),P,"(kPa, kN/m2)",T,(oC)
1,P,"(kN/m2, kPa)",T,(oC),P,"(kN/m2, kPa)"
2,dU,(centiPoise),SV,(m3/kg),ro,(kg/m3)
3,kU,(10-6 m2/s),ro,(kg/m3),SV,(10-3 m3/kg)
4,alpha,(10-3 1/K),HL,(kJ/kg),Cp,(kJ/(kg K))
5,HL,NaN,HV,(kJ/kg),sH,(kJ/(kg K))
6,NPr,NaN,HG,(kJ/kg),NaN,NaN
7,NaN,NaN,sHS,(kJ/kgK),NaN,NaN


In [6]:
#import input data for tower model
indat = pd.read_excel('data input.xlsx',sheetname='input',index_col='Variable')
indat.head()

,Input,Units,Description
Variable,,,
RR,40000.000000,gpm,Recirc Rate
TR,89.000000,deg F,Return Temp
TRC,31.666667,deg C,Return Temp
TS,73.000000,deg F,Supply Temp
TSC,22.777778,deg C,Supply Temp


In [7]:
#import CO2 solubility data for tower model
CO2 = pd.read_excel('properties database.xlsx',sheetname='CO2sol',index_col='T')
CO2.head()

,g/kg
T,
0,0.440
20,0.280
40,0.210
60,0.015
80,0.010


In [8]:
#Interpolation function

def interp(value,x,y):
    return np.interp(value,x,y)

In [9]:
#Create mass flow rate array
M = np.array([indat.loc['M0']['Input'],indat.loc['M1']['Input'],
                indat.loc['M2']['Input'],indat.loc['M3']['Input'],indat.loc['M4']['Input']])
#Create temperature array
T = np.array([indat.loc['T0']['Input'],indat.loc['T1']['Input'],indat.loc['T2']['Input'],
                indat.loc['T3']['Input'],indat.loc['T4']['Input']])
#Mass fraction array
COC = indat.loc['COC']['Input']
X = np.array([1,COC,COC,COC,0])

#System volume
Vsys = indat.loc['V']['Input']

#Create array for specific enthalpy 
H = np.zeros(5)
for i in range(len(T)):
    H[i]=(interp(T[i],steam.index,steam['HL']))
H[4] = interp(T[4],steam.index,steam['HG'])

In [28]:
#  Mass Balance equations
# 1 M0+M3=M1+M2+M4
# 2 M0*H0+M3*H3=M1*H1+M2*H2+M4*H4
# 3 MO*X0+M3*X3=M1*X1+M2*X2+M4*X4
# Solver format a*x = b 
def massbal(COC,T2,T3,RR):

    M[3] = RR
    M[2] = RR
    X[1] = COC
    X[2] = COC
    X[3] = COC
    H[2] = interp(T2,steam.index,steam['HL'])
    H[3] = interp(T3,steam.index,steam['HL'])
    
    
    b1 = M[2]-M[3]
    b2 = M[2]*H[2]-M[3]*H[3]
    b3 = M[2]*X[2]-M[3]*X[3]

    a1 = (1,-1,-1)
    a2 = (H[0],-H[1],-H[4])
    a3 = (X[0],-X[1],-X[4])

    A = np.array([a1,a2,a3])
    B = np.array([b1,b2,b3])

    sol = np.linalg.lstsq(A,B)

    M[0] = sol[0][0]
    M[1] = sol[0][1]
    M[4] = sol[0][2]
    #Convert kg/min to gal/min 
    V = M*0.26
    return M
massbal(COC,T[2],T[3],M[3])

776.26365912841402

In [89]:
#Decay of slug doses
CA0 = 500 #Input PPM

#Time series over the course of a day, minutes 
ts = np.linspace(0,200,50)
#Outlet flow rate

Mo = lambda COCx: massbal(COCx,T[2],T[3],M[3])[1]


def dCa(CA,t,COCx):
    dCA_dt = (-Mo(COCx)*CA)/(Vsys/.26)
    return dCA_dt

def sol(COCx,t):
    return odeint(dCa,CA0,t*60,args=(COCx,))


#def f(x):
#    return sol(x.name,x.index)
COC1 = np.linspace(2,10,10)


soldf = pd.DataFrame(index=ts,columns=COC1)
#soldf1 = soldf.stack().to_frame().apply(lambda x: sol(x.name[0],x.index[1]))
soldf1 = soldf.applymap(lambda x: sol(x.name,x.index))

AttributeError: ("'float' object has no attribute 'name'", u'occurred at index 2.0')

In [ ]:
s